In [0]:
path = "drive/My Drive/Colab Notebooks/466-proj2/"
f = open(path + 'readme.txt', 'r')
print(f.read())
f.close()

In [2]:
FILE = "committee_utterances.tsv"

import pandas as pd

df = pd.read_csv(path + FILE, sep='\t')
df

,vid,fileid,cid,c_name,c_house,hid,position,pid,diarization_id,last,first,start,end,utterance_order,text
0,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,6,24,1,"Go ahead and call the hearing to order, our sp..."
1,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,24,41,2,But also to the communities and it's had other...
2,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,41,56,3,limited water resources are all challenges tha...
3,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,56,71,4,"And we've invited expert witnesses, who'll dis..."
4,4527,8xcAFOvPC50,2,Agriculture,Assembly,648,1,95,1,Galgiani,Cathleen,71,88,5,Drought is certainly a critical issue for Cali...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30858,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1485,1494,555,"Current vote count is 9-3, and that bill is ou..."
30859,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1494,1498,556,Anderson? Mitchell? (aye) Moorlach?
30860,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,103,1,Hernandez,Ed,1504,1512,557,Current vote count is 9-2. That bill is out. I...
30861,2858,nMgNQIdoyHs,148,Public Health and Developmental Services,Senate,497,4,2998,4,Secretary,Committee,1512,1521,558,Morrell? (no) Anderson? Mitchell? (aye) 9-3.


In [87]:
#Select a random 25% (1/4) of the content
import random
from pprint import pprint

records = list(df.text)
number_selected = len(records) // 16  #TODO CHANGE THIS VALUE TO 4
selected_records = random.sample(records, number_selected) #get random sample of number_selected records without replacement

utterances = [record for record in selected_records] #get all the utterance fields only (column #15) for clustering
print("total number of points:",len(utterances))

total number of points: 1928


In [39]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.corpus import wordnet 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) 
punctuation = "-!?;:\"\'.,"

def getPOS(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def getFeatures(text):
    features = {}
    tokens = nltk.word_tokenize(text.lower())
    tot_cnt = len(tokens)
    for word in tokens:        
        if word not in stop_words and word not in punctuation:
            w = wordnet_lemmatizer.lemmatize(word, getPOS(word))
            if w not in features:
                features[w] = 1/float(tot_cnt)
            else:
                features[w] += 1/float(tot_cnt)
    return features


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [52]:
data = df['text']
data = data[:2]

getFeatures(data[0])
data

0    Go ahead and call the hearing to order, our sp...
1    But also to the communities and it's had other...
Name: text, dtype: object

In [0]:
import numpy as np
import random, sys
from collections import Counter

max_t = 2   # max itterations (ignores threshold) (-1 to unset)
k = 2
M = []
M.append([0 for i in range(0, k)])

def initCentroids(vec_data, k):
    centroidID = np.random.permutation(len(vec_data))[:k]
    return [id for id in centroidID]

def calculateDistance(v_dict, u_dict):
    v = list(v_dict.values())
    u = list(u_dict.values())
    total = 0
    for i in range(0, len(v)):
        total += ((v[i] - u[i]) ** 2)
    return np.sqrt(total)


# takes a vector dictionary and a set of all the keys
# returns a new vector dictionary with all the keys
def reshapeVectDict(vect_dict, keys):
    old_shape = len(vect_dict)
    for key in keys:
        if key not in vect_dict:
            vect_dict[key] = 0.0
    return vect_dict

# For a given point, find the closest centroid from the list, return centroid's index
def closestCentroid(point, centroids):
    # Get all the keys (words) in the point and centroids
    # This will be used to transform them to the same dimension so we can do math
    all_keys = []
    all_keys.extend(point)        
    for centroid in centroids:        
        all_keys.extend(centroid.keys())
    all_keys = set(all_keys)

    closestCentroid = 0
    minDist = sys.maxsize
    point_reshape = reshapeVectDict(point, all_keys)

    for centroid in centroids:
        centroid_reshape = reshapeVectDict(centroid, all_keys)
        dist = calculateDistance(point_reshape, centroid_reshape)

        if (dist < minDist):
            minDist = dist
            closestCentroid = centroids.index(centroid)

    return closestCentroid


def k_means(vec_data, k, e):
    t = 0
    M[t] = initCentroids(vec_data, k)

    if max_t >= 0:        
        while (t < max_t):
            t += 1
            C = []
            for i in range(0, k):
                C.append([])
            # Centroid assignment
            for point in vec_data:
                clusterID = closestCentroid(point, M[t])
                

In [0]:
data = selected_records
vec_data = [getFeatures(text) for text in data]

In [0]:
k_means(vec_data, 2, 1)

In [91]:
# M holds the centroid IDs
M

[[1609, 48]]

In [204]:
# vec_data contains all the vectorized data
# initialize centroids
centroids = [vec_data[c] for c in initCentroids(vec_data, 2)]
pprint(centroids)

[{'7-0': 0.14285714285714285, 'bill': 0.14285714285714285},
 {"'re": 0.02564102564102564,
  'also': 0.02564102564102564,
  'bill': 0.05128205128205128,
  'california': 0.02564102564102564,
  'coalition': 0.02564102564102564,
  'cosponsor': 0.02564102564102564,
  'create': 0.02564102564102564,
  'de': 0.02564102564102564,
  'healthy': 0.02564102564102564,
  'incentive': 0.02564102564102564,
  'la': 0.02564102564102564,
  'latino': 0.02564102564102564,
  'nutrition': 0.02564102564102564,
  'program': 0.05128205128205128,
  'rebecca': 0.02564102564102564,
  'rosa': 0.02564102564102564,
  'strong': 0.02564102564102564,
  'support': 0.02564102564102564,
  'would': 0.02564102564102564}]


In [212]:
point = vec_data[0]

centroidID = closestCentroid(point, centroids)
print("point is closest to the centroid, ", centroidID)
print("point: ")
print(point)
print("closest centroid: ")
print(centroids[centroidID])
print("other centroids: ")
print([c for c in centroids if not centroids.index(c) == centroidID])

point is closest to the centroid,  1
point: 
{'family': 0.020833333333333332, 'school': 0.020833333333333332, 'program': 0.020833333333333332, 'allows': 0.020833333333333332, 'job': 0.020833333333333332, 'integrate': 0.020833333333333332, 'community': 0.041666666666666664, 'immigrant': 0.020833333333333332, 'low': 0.020833333333333332, 'income': 0.020833333333333332, 'like': 0.020833333333333332, "'s": 0.020833333333333332, 'really': 0.020833333333333332, 'important': 0.020833333333333332, 'able': 0.020833333333333332, 'get': 0.020833333333333332, 'poverty': 0.020833333333333332, 'thank': 0.020833333333333332, 'much': 0.020833333333333332, "'ve": 0.0, 'a': 0.0, 'b': 0.0, 'c': 0.0, 'kind': 0.0, 'continue': 0.0, 'ensures': 0.0, 'health': 0.0, '50': 0.0, 'four': 0.0, 'experienced': 0.0, 'aye': 0.0, 'many': 0.0, 'reason': 0.0, 'support': 0.0, 'regional': 0.0, 'attention': 0.0, 'think': 0.0, 'na': 0.0, 'provide': 0.0, 'economically': 0.0, 'sure': 0.0, 'people': 0.0, 'respectfully': 0.0, 'sc

In [197]:
total = 0
total += (1 - 2) ** 2
total

1

In [101]:

# Python program to combine two dictionary 
# adding values for common keys 
from collections import Counter   
  
# adding the values with common key 
          
Cdict = Counter(dict_1) + Counter(dict_2) 
print(Cdict)

Counter({'get': 0.13194444444444445, 'thank': 0.13194444444444445, "'ve": 0.1111111111111111, 'community': 0.041666666666666664, 'family': 0.020833333333333332, 'school': 0.020833333333333332, 'program': 0.020833333333333332, 'allows': 0.020833333333333332, 'job': 0.020833333333333332, 'integrate': 0.020833333333333332, 'immigrant': 0.020833333333333332, 'low': 0.020833333333333332, 'income': 0.020833333333333332, 'like': 0.020833333333333332, "'s": 0.020833333333333332, 'really': 0.020833333333333332, 'important': 0.020833333333333332, 'able': 0.020833333333333332, 'poverty': 0.020833333333333332, 'much': 0.020833333333333332})


In [127]:
keys = ['a','b','c']
dict_2 = {i:0 for i in keys}
print(dict_2)
dict_1 = {}
dict_1['a'] = 23

Cdict = Counter(dict_1) + Counter(dict_2) 
Cdict

{'a': 0, 'b': 0, 'c': 0}


Counter({'a': 23})

In [114]:
all_keys = []
for centroid in centroids:
    all_keys.extend(centroid.keys())
all_keys = set(all_keys)

vec1 = []
vec2 = []
for key in all_keys:
    if key not in dict_1:
        vec1.append(0)
    else:
        vec1.append(dict_1[key])
    if key not in dict_2:
        vec2.append(0)
    else:
        vec2.append(dict_2[key])

print(vec1)
print(vec2)
a = np.matrix(vec1)
b = np.matrix(vec2)
ret = a - b
print(ret)

[0.020833333333333332, 0.0, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.041666666666666664, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332, 0.020833333333333332]
[0.1111111111111111, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[[-0.09027778 -0.11111111  0.02083333  0.02083333  0.02083333  0.02083333
   0.02083333  0.04166667 -0.09027778  0.02083333  0.02083333  0.02083333
   0.02083333  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333
   0.02083333  0.02083333]]


In [116]:
# Python Program illustrating  
# numpy.mean() method    
import numpy as np 
    
  
# 2D array  
arr = [[2,10],   
       [2,10]]
       
    
# mean of the flattened array  
print("\nmean of arr, axis = None : ", np.mean(arr))  


mean of arr, axis = None :  6.0
